In [ ]:
from queue import Queue
import random
from threading import Thread
import time


def producer(queue: Queue[int | None]):
    for i in range(10):
        time.sleep(random.uniform(0, 0.5))
        queue.put(i)
        print("生产者：", i)
    queue.put(None)


def consumer(queue: Queue[int | None]):
    while True:
        try:
            data = queue.get()
            if data is None:
                break
            time.sleep(random.uniform(0.5, 1.0))
            print(f"[consumer] 消费 {data}")
        finally:
            queue.task_done()


# 防止请求洪峰把内存打爆
queue = Queue(maxsize=3)
t1 = Thread(target=producer, args=(queue,))
t2 = Thread(target=consumer, args=(queue,))

t1.start()
t2.start()

t1.join()
t2.join()
print("所有任务完成")


生产者： 0
生产者： 1
生产者： 2
[consumer] 消费 0
生产者： 3
生产者： 4
[consumer] 消费 1
生产者： 5
[consumer] 消费 2
生产者： 6
[consumer] 消费 3
生产者： 7
[consumer] 消费 4
生产者： 8
[consumer] 消费 5
生产者： 9
[consumer] 消费 6
[consumer] 消费 7
[consumer] 消费 8
[consumer] 消费 9
所有任务完成


In [ ]:
from queue import Queue
import threading
import time
import random


def producer(q: Queue[str | None], data: list[str]):
    for d in data:
        q.put(d)
        print(f"【生产者】创建任务：{d}")
    q.put(None)
    print(f"【生产者】创建任务完成")


def consumer(q: Queue[str | None], results: list[str]):
    """
    问题分析：q.task_done() 会在 q.get() 还没成功时也执行（隐性崩溃点）

    """
    while True:
        try:
            task = q.get()
            if task is None:
                print("【消费者】消费任务完成")
                return results
            time.sleep(random.uniform(0.1, 0.5))
            results.append(f"done:{task}")
            print(f"【消费者】消费任务：{task}")
        finally:
            q.task_done()


def run_tasks(data: list[str], maxsize: int = 2):
    q: Queue[str] = Queue(maxsize)
    results: list[str] = []
    t1 = threading.Thread(target=producer, args=(q, data))
    t2 = threading.Thread(target=consumer, args=(q, results))

    t1.start()
    t2.start()

    t1.join()
    # ✅ 等待所有 put 的条目（包含 None）都被 task_done
    # 阻塞至队列中所有的元素都被接收和处理完毕。
    q.join()
    t2.join()
    """
        t1.join() 等生产结束
        q.join() 等队列计数清零（包括 None）
        t2.join() 等消费者退出
    """

    assert results == ["done:A", "done:B", "done:C"], f"results：{results} != {data}"


data = ["A", "B", "C"]
QUEUE_MAXSIZE = 2
run_tasks(data, QUEUE_MAXSIZE)


【生产者】创建任务：A
【生产者】创建任务：B
【生产者】创建任务：C
【消费者】消费任务：A
【生产者】创建任务完成
【消费者】消费任务：B
【消费者】消费任务：C
【消费者】消费任务完成


如果 q.get() 这一行在某些异常情况下没拿到 item（例如线程被打断、未来你改成 get(timeout=...) 抛 Empty），task_done() 就会“多减一次计数”，直接触发 ValueError: task_done() called too many times

In [7]:
import logging
import threading
import time
from queue import Queue, Empty

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(threadName)s %(levelname)s | %(message)s",
)


def consumer_wrong(q: Queue[str]) -> None:
    """错误示范：get 超时拿不到 item，也会执行 task_done() -> 计数被多减"""
    while True:
        try:
            logging.info("设置元素超时 0.2s")
            item = q.get(timeout=0.2)  # 队列为空 -> 抛 Empty
            logging.info("消费者获取元素=%r", item)

            if item is None:
                logging.info("任务已完成")
                return

        except Empty:
            logging.warning("队列已空，等待中...")

        finally:
            # ❌ 错：即使没有成功 get 到 item，也会 task_done()
            logging.info("消费者 完成一个任务")
            q.task_done()  # 会直接 ValueError: task_done() called too many times


def main() -> None:
    q: Queue[str] = Queue()
    # 故意什么都不 put，让 consumer 立即 timeout
    t = threading.Thread(target=consumer_wrong, args=(q,), name="consumer-wrong")
    t.start()
    t.join()


main()


2026-01-27 02:48:01,964 consumer-wrong INFO | 设置元素超时 0.2s
2026-01-27 02:48:02,195 consumer-wrong WARNING | 队列已空，等待中...
2026-01-27 02:48:02,196 consumer-wrong INFO | 消费者 完成一个任务
Exception in thread consumer-wrong:
Traceback (most recent call last):
  File "C:\Users\coin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\coin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\coin\AppData\Local\Temp\ipykernel_72044\313262045.py", line 30, in consumer_wrong
  File "C:\Users\coin\AppData\Local\Programs\Python\Python312\Lib\queue.py", line 75, in task_done
    raise ValueError('task_done() called too many times')
ValueError: task_done() called too many times


In [1]:
from queue import Queue
import threading
import time
import random


def producer(q: Queue[str | None], data: list[str]) -> None:
    for d in data:
        q.put(d)  # ✅ 队列满会阻塞 -> 背压生效
        print(f"【生产者】创建任务：{d}")
    q.put(None)
    print("【生产者】创建任务完成")


def consumer(q: Queue[str | None], results: list[str]) -> None:
    while True:
        # 设置获取状态
        got_item = False
        task: str | None = None
        try:
            task = q.get()
            # 获取到元素后才设置状态
            got_item = True

            if task is None:
                print("【消费者】消费任务完成")
                return

            time.sleep(random.uniform(0.1, 0.5))
            results.append(f"done:{task}")
            print(f"【消费者】消费任务：{task}")
        finally:
            # ✅ 只对“成功 get 到的 item”调用 task_done
            if got_item:
                q.task_done()


def run_tasks(data: list[str], maxsize: int = 2) -> list[str]:
    q: Queue[str | None] = Queue(maxsize=maxsize)
    results: list[str] = []

    t1 = threading.Thread(target=producer, args=(q, data))
    t2 = threading.Thread(target=consumer, args=(q, results))

    t1.start()
    t2.start()

    t1.join()
    q.join()  # ✅ 等待所有 put 的条目（包含 None）都被 task_done
    t2.join()

    return results
